In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

from pathlib import Path

import logging
logging.basicConfig(level=logging.INFO)

import numpy

In [3]:
import structlib
from structlib import AtomCell, SimpleAtoms
from structlib.visualize import show_atoms_3d
from structlib.disloc import disloc_loop_z, disloc_poly_z, disloc_edge

In [4]:
cell = structlib.make.wurtzite('AlN', 3.13, 5.02, cell='prim')
show_atoms_3d(cell);

In [5]:
from structlib.cell import plane_to_zone
from structlib.util import reduce_vec

plane_to_zone(cell.metric, [1., 0., 0.])

In [5]:
cell: AtomCell = AtomCell.read(Path('AlN-ortho.xsf'))

show_atoms_3d(cell);

print(str(cell.cell_size))

cell = cell.repeat((8, 5, 5), explode=True)
#cell = cell.repeat(3, explode=True)

show_atoms_3d(cell);

In [83]:
edge = disloc_edge(cell, center=[0., 12., 12.], b=[0., 0., 5.02/2.], t=[1., 0., 0.], cut='add')
show_atoms_3d(edge);

In [7]:
frame = cell.get_atoms('local').with_occupancy(1 - 1000. / len(cell))  # average 1000 vacancies
frame = frame.apply_occupancy()
print(f"Removed {len(cell) - len(frame)} atoms")

vacant = cell._replace_atoms(frame, 'local')

vacant.write('vacancies2.xsf')
vacant.write('vacancies2.lmp')

In [48]:
from structlib.transform import AffineTransform
import polars

faults = cell._replace_atoms(cell.atoms.transform(AffineTransform.translate(0., 5.421/3, 0.), (polars.col('z') > 11.) & (polars.col('z') < 26.)))
faults = faults.wrap()
show_atoms_mpl_2d(faults.crop_atoms(x_min=16., x_max=30.), zone=[1,0,0]);

faults.write('fault_pair.xsf')
faults.write('fault_pair.lmp')

In [13]:
#poly = [[10., 10.], [40., 10.], [10., 40.], [40., 40.]]

#loop = disloc_poly_z(cell, [0., 5.421/3, 5.02/2], poly, center=[0., 0., 20.])

loop = disloc_loop_z(cell, center=[3.*24., 3.*24., 3.*20.], b=[0., 5.421/3, 5.02/2.,], loop_r=15.)

show_atoms_3d(loop.crop_atoms(x_min=3.*16., x_max=3.*30.));
#show_atoms_3d(loop);
#loop

In [5]:
from structlib.transform import LinearTransform

LinearTransform.align_to([1., 1., 1.], [0., 0., 1.], [0., 1., 0.], [1., 0., 1.])

In [14]:
from structlib.visualize import show_atoms_mpl_2d

show_atoms_mpl_2d(loop, zone=[1., 0., 0.], horz=[0., 1., 0.]);

In [15]:
loop.write("loop_extrinsic_large.xsf")
loop.write("loop_extrinsic_large.lmp")
#loop.write("loop_extrinsic_as_hour.xsf")
#loop.write("loop_extrinsic_as_hour.lmp")